# Natural Instructions

In [3]:
files = list(os.listdir("/share/jiziwei/natural-instructions-2.8/tasks"))
print(len(files))
all_num = 0

tasks = []
with open('/share/jiziwei/natural-instructions-2.8/selected_tasks.txt') as f:
    for l in f.readlines():
        if "#" not in l:
            tasks.append(l.strip())
print(tasks)           
# no_tasks = set()
for i, file in tqdm(enumerate(files)):
    with open(f"/share/jiziwei/natural-instructions-2.8/tasks/{file}") as f:
        if not file.endswith("json"):
            continue
        # try:
        data = json.load(f)
        if any([c not in tasks for c in data['Categories']]):
            continue
        assert len(data['Categories']) == 1
        category = "_".join(data['Categories'][0].strip().split())
        if file in ["task288_gigaword_summarization.json"]:
            data['Definition'] = [data['Definition'][1]]
        if len(data['Definition']) > 1:
            print(file, data['Definition'])
        assert len(data['Definition']) == 1
        instruction = data['Definition'][0]
        instances = data['Instances']
        X_train, X_test_val, y_train, y_test_val = train_test_split(instances, list(range(len(instances))), test_size=0.2, random_state=42)
        X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)
        # except:
        #     print("wrong", file)
        file = file[:-5]
        os.makedirs(f"/share/jiziwei/natural-instructions-2.8/processed/{category}/{file}", exist_ok=True)
        with jsonlines.open(f"/share/jiziwei/natural-instructions-2.8/processed/{category}/{file}/train.jsonl", 'w') as writer:
            for line in X_train:
                line2 = {"id": line["id"], 
                         "question": f"{instruction}\n{line['input']}", 
                         "answer": line["output"], }
                writer.write(line2)
                
        with jsonlines.open(f"/share/jiziwei/natural-instructions-2.8/processed/{category}/{file}/test.jsonl", 'w') as writer:
            for line in X_test:
                line2 = {"id": line["id"], 
                         "question": f"{instruction}\n{line['input']}", 
                         "answer": line["output"], }
                writer.write(line2)

        with jsonlines.open(f"/share/jiziwei/natural-instructions-2.8/processed/{category}/{file}/val.jsonl", 'w') as writer:
            for line in X_val:
                line2 = {"id": line["id"], 
                         "question": f"{instruction}\n{line['input']}", 
                         "answer": line["output"], }
                writer.write(line2)
                

1614
['Question Answering', 'Overlap Extraction', 'Program Execution', 'Text to Code', 'Data to Text', 'Code to Text', 'Translation', 'Paraphrasing', 'Explanation', 'Title Generation', 'Dialogue Generation', 'Grammar Error Correction', 'Number Conversion', 'Summarization', 'Sentence Compression', 'Preposition Prediction', 'Question Understanding', 'Answerability Classification', 'Text Quality Evaluation', 'Toxic Language Detection', 'Coreference Resolution', 'Answer Verification', 'Coherence Classification', 'Mathematics', 'Spelling Error Detection', 'Grammar Error Detection', 'Ethics Classification', 'Spam Classification', 'Linguistic Probing', 'Text Categorization', 'Commonsense Classification', 'Word Semantics', 'Text Matching', 'Information Extraction', 'Textual Entailment', 'Sentiment Analysis', 'Sentence Composition', 'Sentence Ordering', 'Paper Review', 'Language Identification', 'Stereotype Detection', 'Gender Classification', 'Section Classification', 'Dialogue Act Recognition

0it [00:00, ?it/s]

In [ ]:
results